In [108]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [109]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [110]:
df = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')
df

In [111]:
df.info()
#no null values so no need to worry about them :)

# EDA
Detecting Outliers

In [112]:
numeric_cols = df.drop(['cp','thall','output','fbs','restecg','exng'],axis=1).copy()
plt.show()
#numeric_cols
plt.figure(figsize=(16,10))
for i in range(len(numeric_cols.columns)):
   plt.subplot(2,4,i+1)
   sns.boxplot(numeric_cols[numeric_cols.columns[i]])
plt.show()    
    

In [113]:
plt.figure(figsize = (8,10))
sns.heatmap(numeric_cols.corr(),cmap='BrBG')
plt.title('Co-Relation Heat Map')
plt.show()

In [114]:
plt.figure(figsize=(6,6))
plt.pie(df['output'].value_counts(),labels=['Heart Disease','No Heart Disease'],colors = ['mediumpurple','rebeccapurple'],autopct ='%.2f%%' )
plt.title('Distribution')
plt.show()

In [115]:
plt.figure(figsize=(7,7))
plt.pie(df['sex'].value_counts(),labels = ['Male','FeMale'],autopct='%.2f%%')
plt.title('Gender Distribution')
plt.show()

# Pre Processing

In [116]:
def HotOneEncode(df, prefix_dict):
    df = df.copy()
    for col,pre in prefix_dict.items():
        dummy = pd.get_dummies(df[col],prefix = pre)
        df = pd.concat([df,dummy],axis=1)
        df = df.drop(col,axis=1)
    return df

In [117]:
def PreProcess(data,scaler):
    #Encoding
    categorical_cols = ['cp','slp','thall']
    data = HotOneEncode(data,dict(zip(categorical_cols,['CP','SLP','THL'])))
    #SPLITTING
    x = data.drop('output',axis =1).copy()
    y = data['output'].copy()
    #Scaling 
    x = pd.DataFrame(scaler.fit_transform(x),columns= x.columns)
    return x,y

In [118]:
x,y = PreProcess(df,StandardScaler())

In [119]:
#{colum: len(x[colum].unique()) for colum in x.columns}

In [120]:
x

In [121]:
y

# Training

Logistic Regression

In [122]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=40)
model = LogisticRegression()
model.fit(x_train,y_train)
print("Accuracy: "+ str(round(model.score(x_test,y_test)*100,2)))